In [ ]:
!pip install transformers datasets torch pandas openpyxl
!pip install fsspec==2024.10.0
!pip uninstall gcsfs -y


  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [ ]:
import pandas as pd

# Load the Excel file and display the column names for each sheet
excel_path = '/content/The English Tree.xlsx'
sheet_names = pd.ExcelFile(excel_path).sheet_names

for sheet in sheet_names:
    df = pd.read_excel(excel_path, sheet_name=sheet)
    print(f"Sheet '{sheet}' column names: {df.columns.tolist()}")



Sheet 'grade8' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade7' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade6' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade5' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade4' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade3' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-Response Eupheus', 'Context']
Sheet 'grade2' column names: ['Series', 'Grade', 'Chapter name ', 'Question', 'Answer', 'chatbot-response', 'Chatbot-R

In [ ]:
import pandas as pd

# Load your uploaded Excel file
excel_path = '/content/The English Tree.xlsx'
sheet_names = pd.ExcelFile(excel_path).sheet_names

# Prepare the dataset in question-answer format
data = []

for sheet in sheet_names:
    df = pd.read_excel(excel_path, sheet_name=sheet)
    for idx, row in df.iterrows():
        question = row['Question']  # Column with questions
        answer = row['Answer']      # Column with answers
        data.append({"input": f"Question: {question} Answer:", "output": answer})

# Convert to a DataFrame for easier handling
dataset = pd.DataFrame(data)

# Inspect the data
dataset.head()


,input,output
0,Question: What did Kino’s and Jiya’s fathers ...,"Kino’s father was a farmer, Jiya’s father was..."
1,Question: What did Jiya think about the sea? ...,Jiya thought the sea was the enemy.
2,"Question: What did Jiya and Kino do on hot, s...",Kino and Jiya enjoyed swimming far out toward...
3,Question: Why did Kino think their home was s...,Kino thought their home was safe because they...
4,Question: What did Kino see one summer day? A...,"One summer day, Kino saw the unrest under the..."


In [ ]:
# Check the total number of entries from all sheets
print(f"Total entries loaded: {len(dataset)}")


Total entries loaded: 858


In [ ]:
excel_path = '/content/The English Tree.xlsx'
sheet_names = pd.ExcelFile(excel_path).sheet_names

# Display the count of sheets
print(f"Total number of sheets: {len(sheet_names)}")


Total number of sheets: 8


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for distilgpt2 and set the padding token
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as the padding token

# Tokenize the data
def tokenize_function(row):
    # Ensure input and output are strings, replace NaNs or None with empty strings
    input_text = str(row['input']) if pd.notnull(row['input']) else ""
    output_text = str(row['output']) if pd.notnull(row['output']) else ""

    # Tokenize the input text and output text with padding and truncation
    inputs = tokenizer(input_text, padding='max_length', truncation=True, max_length=128)
    labels = tokenizer(output_text, padding='max_length', truncation=True, max_length=128)
    inputs['labels'] = labels['input_ids']
    return inputs

# Apply tokenization to each row and convert to a list of dictionaries
tokenized_data = dataset.apply(tokenize_function, axis=1).tolist()


In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import Dataset

# Load the pre-trained model
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Convert the tokenized data into a Hugging Face Dataset
train_data = [{"input_ids": x['input_ids'], "labels": x['labels']} for x in tokenized_data]
train_dataset = Dataset.from_list(train_data)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.094100
1000,0.886900


TrainOutput(global_step=1287, training_loss=0.952306336919374, metrics={'train_runtime': 7091.1302, 'train_samples_per_second': 0.363, 'train_steps_per_second': 0.181, 'total_flos': 84072229502976.0, 'train_loss': 0.952306336919374, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [1]:
# Install required packages
!pip install sentence-transformers faiss-cpu

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load your dataset
file_path = '/content/The English Tree.xlsx'
excel_data = pd.ExcelFile(file_path)

# Combine all sheets
all_data = pd.concat([excel_data.parse(sheet) for sheet in excel_data.sheet_names])

# Extract questions and answers
questions = all_data['Question'].astype(str).tolist()
answers = all_data['Answer'].astype(str).tolist()

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the questions as embeddings
question_embeddings = model.encode(questions, convert_to_tensor=False)

# Convert embeddings to a numpy array for faiss
question_embeddings = np.array(question_embeddings)

# Set up faiss index
index = faiss.IndexFlatL2(question_embeddings.shape[1])  # L2 distance index
index.add(question_embeddings)

def get_answer(input_question):
    # Encode the input question
    input_embedding = model.encode([input_question])

    # Retrieve the nearest question
    _, index_result = index.search(np.array(input_embedding).astype('float32'), 1)
    best_match_index = index_result[0][0]

    # Return the answer corresponding to the nearest question
    return answers[best_match_index]

# Test the retrieval with an example question
test_question = "Why did Rishi take two plants?"
print("Answer:", get_answer(test_question))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 18.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Answer: Rishi took two plants because he wanted to plant one in the school garden and one in a pot at home.


In [12]:
def get_answer(input_question):
    # Encode the input question
    input_embedding = model.encode([input_question])

    # Retrieve the nearest question
    _, index_result = index.search(np.array(input_embedding).astype('float32'), 1)
    best_match_index = index_result[0][0]

    # Return the answer corresponding to the nearest question
    return answers[best_match_index]

# Test the retrieval with an example question
test_question = "Why was Mamta  given the National Bravery Award?"  # Replace with an actual question from your data
print("Answer:", get_answer(test_question))

Answer: Mamta saved her sister from the crocodile. She showed her bravery and hence she was awarded.


In [1]:
!pip install streamlit ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 744.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Streamlit App
st.title("Question Answering Bot")

# User Inputs
grade = st.selectbox("Select Grade", ["Grade 1", "Grade 2", "Grade 3", "Grade 4", "Grade 5", "Grade 6", "Grade 7", "Grade 8"])
series = st.text_input("Enter Series")
question = st.text_input("Enter Question")

# Button to Get Answer
if st.button("Get Answer"):
    input_text = f"Grade: {grade}, Series: {series}, Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=128)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    st.write("Answer:", answer)


Writing app.py


In [4]:

!pip install pyngrok

In [12]:
!npm install -g localtunnel



added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [13]:
!npm audit fix --force

npm WARN using --force Recommended protections disabled.
npm ERR! code ENOLOCK
npm ERR! audit This command requires an existing lockfile.
npm ERR! audit Try creating one first with: npm i --package-lock-only
npm ERR! audit Original error: loadVirtual requires existing shrinkwrap file

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-11-14T06_10_34_884Z-debug-0.log


In [16]:
!curl -sSL https://railway.app/install.sh | sh



> Installing railway, please wait…
                   .
         /^\     .
    /\   "V"
   /__\   I      O  o             
  //..\\  I     .                             Poof!
  \].`[/  I
  /l\/j\  (]    .  O
 /. ~~ ,\/I          .               Railway is now installed
 \\L__j^\/I       o               Run `railway help` for commands
  \/--v}  I     o   .
  |    |  I   _________                    
  |    |  I c(`       ')o            
  |    l  I   \.     ,/
_/j  L l\_!  _//^---^\\_ 



In [22]:
!
!railway help




Interact with Railway via CLI

Usage: railway [OPTIONS] <COMMAND>

Commands:
  add          Add a service to your project
  completion   Generate completion script
  connect      Connect to a database's shell (psql for Postgres, mongosh for MongoDB, etc.)
  deploy       Provisions a template into your project
  domain       Generates a domain for a service if there is not a railway provided domain
  docs         Open Railway Documentation in default browser
  down         Remove the most recent deployment
  environment  Change the active environment
  init         Create a new project
  link         Associate existing project with current directory, may specify projectId as an argument
  list         List all projects in your Railway account
  login        Login to your Railway account
  logout       Logout of your Railway account
  logs         View a deploy's logs
  open         Open your project dashboard
  run          Run a local command using variables from the active environment

In [27]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [33]:
!ls -R /content/drive/MyDrive/Colab\ Notebooks



'/content/drive/MyDrive/Colab Notebooks':
ingestion_excel_to_bq.ipynb	 PdfandExcelSummarization.ipynb  Untitled1.ipynb
ingestion_new_experiement.ipynb  retrevial_distilgpt2.ipynb
json_to_jsonl.ipynb		 Untitled0.ipynb


In [43]:
%cd /content/drive/MyDrive/Colab Notebooks/



/content/drive/MyDrive/Colab Notebooks


In [40]:
!ls

ingestion_excel_to_bq.ipynb	 PdfandExcelSummarization.ipynb  Untitled1.ipynb
ingestion_new_experiement.ipynb  retrevial_distilgpt2.ipynb
json_to_jsonl.ipynb		 Untitled0.ipynb


In [25]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [26]:
!git config --global user.name "Dhaatri123"
!git config --global user.email "raodhaatri@gmail.com"


In [44]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/.git/


In [50]:
!git add retrevial_distilgpt2.ipynb



In [51]:
!git commit -m "Initial commit"


[master 22a776c] Initial commit
 1 file changed, 1 insertion(+), 1 deletion(-)


In [52]:
 !git remote add origin https://github.com/Dhaatri123/Streamlit-app.git

error: remote origin already exists.


In [49]:
!git push -u origin master


fatal: could not read Username for 'https://github.com': No such device or address
